# Practical session: Building interactive apps with Streamlit

[Streamlit](https://streamlit.io/) is a faster way to build and share data apps in Python. You can run your script in the command line as follows:

`streamlit run your_script.py [-- script args]`

If installed and run locally, it is similar to a Jupyter notebook in that your app can be viewed in a browser, and the server can be stopped with `Ctrl+C`.

As we are running streamlit within Colab for this practical session, we will use Cloudflare to host the server that runs our app.

## 0. Set up Streamlit and Cloudfare

As a first step, we will install Streamlit and Cloudfare.

In [ ]:
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
--2024-05-15 14:04:53--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.4.1/cloudflared-linux-amd64 [following]
--2024-05-15 14:04:53--  https://github.com/cloudflare/cloudflared/releases/download/2024.4.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting respon

Let's check if we can get the Cloudfare tunnel to work. We will write a simple app to the file `Home.py`. This app displays the square of the number on the slider.

In [ ]:
%%writefile Home.py
import streamlit as st

x = st.slider('Select a value')
st.write(x, 'squared is', x * x)

Writing Home.py


Run the `app.py` file. If running locally, type the code (without the exclamation mark) in the command line, and a new browser should pop up.

In [ ]:
!streamlit run /content/Home.py &>/content/logs.txt &

When running on Google Colab, we have to open a Cloudflare tunnel in the background that displays things we have running on port 8501, which is the port that Streamlit applications are run on by default. Try clicking the link below to see whether your app is successfully hosted there. If there are errors, you may have to install streamlit locally instead.

Also note that since we are running the tunnel without a Cloudflare account, we have no uptime guarantee.

In [ ]:
# Open Cloudflare tunnel
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

nohup: appending output to 'nohup.out'


In [ ]:
# Get link to Cloudflare tunnel
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}" # Display the url where your app will run

Your tunnel url https://achievement-fig-analysts-una.trycloudflare.com


**IMPORTANT:** Only run `streamlit run` once, otherwise you will be running several instances on different ports (by default, the next instances will be on port 8502, 8503, and so on). This may cause your app to not refresh anymore. In that case, you will have to kill all streamlit processes and run it again, using the `kill` command.

To do this, first check the PID of all streamlit processes.

In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      6 ?        00:00:18 node
     17 ?        00:00:03 oom_monitor.sh
     19 ?        00:00:00 run.sh
     20 ?        00:00:02 kernel_manager_
     23 ?        00:00:00 tail
     62 ?        00:00:07 python3 <defunct>
     63 ?        00:00:01 colab-fileshim.
     81 ?        00:00:09 jupyter-noteboo
     82 ?        00:00:07 dap_multiplexer
    190 ?        00:00:33 python3
    217 ?        00:00:16 python3
    775 ?        00:00:14 streamlit
   1710 ?        00:00:08 cloudflared-lin
   8238 ?        00:00:03 language_servic
   8243 ?        00:07:06 node
  25576 ?        00:00:00 sleep
  25577 ?        00:00:00 ps


If there is more than one streamlit process, do:
`!kill -9 {PID}`.

## 1. Building a small chat app

We will start off with a simple chat application that simply echoes what the user input is. When you run the cell below, the file `Home.py` will be overwritten. If you switch to the tab where you have your app opened, the top right should say "$i$ Source file changed.", to which you can click "Rerun". If not, you can click the three dots in the corner and press "Rerun", or just press R (while not in the message box).

In [ ]:
%%writefile Home.py
import streamlit as st

st.title("Echo Bot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What is up?"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    response = prompt
    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})


Overwriting Home.py


**Explanation (also check the API [here](https://docs.streamlit.io/develop/api-reference)):**
*  `st.session_state.messages` is a list that stores the chat history ([session state](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.session_state) is a way to store variable between reruns). Each entry in the list will be a dictionary with the following keys: role (the author of the message), and content (the message content). The author can be either "user" or "assistant" to enable preset styling and avatars. You can pass in a custom string to use as the author name, but this will not be shown in the UI.
*  `st.chat_input`: displays a chat input widget so the user can type in a message. The returned value is the user's input, which is `None` if the user hasn't sent a message yet. You can also pass in a default prompt to display in the input widget.
*  `st.chat_message`: inserts a multi-element chat message container into your app. The first parameter is the name of the message author, and the returned container can contain any Streamlit element, including charts, tables, text, and more. To add elements to the returned container, you can use `with` notation.
* `st.markdown`: writes text formatted as Markdown.

#### 🗒 **TASK** 🗒

Change the reply of the chatbot assistant to whatever you'd like (e.g., add a random emoji at the end, a reverse of the user's input, ...).

## 2. Building a Llama 3 chatbot

Next, we will make use of Llama 3 to generate the responses to user queries. We will use the Llama 3 model hosted on Replicate, a service that lets you run machine learning models with a cloud API. To do this, go to [replicate.com](https://replicate.com) and log in with your GitHub account. Then, copy your token from **Account settings > API tokens**.

Unfortunately, you will only be able to test out a few queries before you are paywalled, but it should be enough to try it out!

In [ ]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 924.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
your_token = ... # paste your token here

import os
if not os.path.exists('.streamlit/'):
    # Create the folder
    os.makedirs('.streamlit/')

file = open('.streamlit/secrets.toml', 'w')
print('REPLICATE_API_TOKEN = "{}"'.format(your_token), file = file)
file.close()


In [ ]:
%%writefile Home.py
import streamlit as st
import replicate
import os

# App title
st.set_page_config(page_title="🦙💬 Llama 3 Chatbot")

# Add sidebar
with st.sidebar:
    st.title('🦙💬 Llama 3 Chatbot')

    # Check token
    if 'REPLICATE_API_TOKEN' in st.secrets:
        replicate_api = st.secrets['REPLICATE_API_TOKEN']
        if not (replicate_api.startswith('r8_') and len(replicate_api)==40):
            st.warning('Your credentials seem to be incorrect!', icon='⚠️')
        else:
            st.success('API key success!', icon='✅')

    os.environ['REPLICATE_API_TOKEN'] = replicate_api

    # Add hyperparameter sliders
    st.subheader('Hyperparameters')
    st.session_state['temperature'] = st.sidebar.slider('temperature', min_value=0.01, max_value=5.0, value=0.1, step=0.01)
    st.session_state['top_p'] = st.sidebar.slider('top_p', min_value=0.01, max_value=1.0, value=0.9, step=0.01)
    st.session_state['max_tokens'] = st.sidebar.slider('max_tokens', min_value=32, max_value=128, value=120, step=8)

# Store LLM generated responses
if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Clear chat history button
def clear_chat_history():
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]
st.sidebar.button('Clear Chat History', on_click=clear_chat_history)

# Function for generating LLAMA3 response. Refactored from https://github.com/a16z-infra/llama2-chatbot
def generate_llama3_response(prompt_input, temperature, top_p, max_tokens):
    pre_prompt = "You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'."
    for dict_message in st.session_state.messages:
        if dict_message["role"] == "user":
            pre_prompt += "User: " + dict_message["content"] + "\n\n"
        else:
            pre_prompt += "Assistant: " + dict_message["content"] + "\n\n"
    output = replicate.run('meta/meta-llama-3-70b-instruct',
                           input={"prompt": f"{pre_prompt} {prompt_input} Assistant: ",
                                  "temperature":temperature, "top_p":top_p, "max_tokens":max_tokens, "repetition_penalty":1})
    return output

# User-provided prompt
if prompt := st.chat_input(disabled=not replicate_api):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)

# Generate a new response if last message is not from assistant
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = generate_llama3_response(prompt, st.session_state['temperature'], st.session_state['top_p'], st.session_state['max_tokens'])
            # Type out response word by word
            placeholder = st.empty()
            full_response = ''
            for item in response:
                full_response += item
                placeholder.markdown(full_response)
            placeholder.markdown(full_response)
    message = {"role": "assistant", "content": full_response}
    st.session_state.messages.append(message)

Overwriting Home.py


The code above includes a sidebar where we allow users to tune [hyperparameters](https://replicate.com/meta/meta-llama-3-70b-instruct/api/schema) of the Llama model. We include three hyperparameters:

* `temperature`: The value used to modulate the next token probabilities.
* `top_p`: A probability threshold for generating the output. If < 1.0, only keep the top tokens with cumulative probability >= top_p (nucleus filtering). Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751).
* `max_length`: The maximum number of words the model should generate as output.

The code was taken from this [blog post]((https://blog.streamlit.io/how-to-build-a-llama-2-chatbot/)) with slight modifications.

## 3. Building a webpage for your Deep learning analysis

In this section, we will turn your deep learning analysis from Practical session 9 into a web application. We will create three pages: the home page, the data loading and preprocessing page, and the model training page. At the bottom of each code cell, you will find the code that is relevant to create the app.

In [ ]:
if not os.path.exists('pages/'):
    # Create the folder to store subpages
    os.makedirs('pages/')

In [ ]:
%%writefile Home.py
import streamlit as st
from tensorflow.python.client import device_lib
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import os

st.set_page_config(
    page_title="Results practical session deep Learning",
    page_icon = "https://cdn-icons-png.flaticon.com/512/6037/6037989.png"
)
st.sidebar.header("Analysis steps")
st.image("https://cdn-icons-png.flaticon.com/512/6037/6037989.png")
st.markdown("# Malaria cell classification ")
st.markdown('''Malaria is a blood disease caused by the Plasmodium parasite, and is transmitted through the bite of the female Anopheles mosquito. The disease is mostly diagnosed by counting parasitized blood cells in a blood smear under a microscope. However, manual cell counting is an exhausting, error-prone procedure. This can negatively affect the quality of the diagnosis. Especially in resource-constrained regions of the world, difficult working conditions lead to poor diagnosis quality.''')
st.markdown('''In this practical session we will develop a deep learning pipeline that will aid in improving malaria diagnosis by automating infected cell counting. To this end we will use the Malaria cell dataset to train a neural network that predicts a cell's infection state based on a microscopy image of it. The microscopy images were acquired using a smartphone attached to a small portable microscope.''')

Overwriting Home.py


In [ ]:
%%writefile pages/1_Data.py
import streamlit as st

from tensorflow.python.client import device_lib
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import os

####### FUNCTIONS #######
def select_device(prefer_gpu=True):
    local_device_protos = device_lib.list_local_devices()
    gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
    if (len(gpus) > 0) and prefer_gpu:
        return gpus[0]
    else:
        return [x.name for x in local_device_protos if x.device_type == 'CPU'][0]

def initialize_data():
  device = select_device(prefer_gpu=True)
  builder = tfds.builder('malaria')
  builder.download_and_prepare()

  with tf.device(device):
    train_ds, test_ds = (
        builder.as_dataset(as_supervised=True, split="train[:80%]"),
        builder.as_dataset(as_supervised=True, split="train[-20%:]")
    )
  return train_ds, test_ds

def plot_10_images(train_ds):
  n_images = 10

  # select some images from the dataset
  images = train_ds.take(n_images)

  fig, axes = plt.subplots(1, n_images, figsize=(20, 5), dpi=100)
  for ax, (image, label) in zip(axes, images):
      ax.imshow(image)
      ax.set_title("Healthy" if label.numpy() == 1 else "Malaria")

  return(fig)

def resize_images(image, label, image_width=40, image_height=40):
    """
    Resizes all images in a dataset to a uniform width and height.
    Also casts the images and labels to the float32 data type.
    """
    return (
        tf.cast(tf.image.resize_with_pad(tf.image.resize(image, (image_width, image_height)), image_width, image_height), tf.float32),
        tf.cast(label, tf.float32)
    )

def minmax_normalization(image, label):
    """Normalizes the pixel value range of an image to [0, 1] by dividing each pixel by the maximum value in each channel."""
    return (
        image / tf.math.reduce_max(tf.reshape(image, [-1, image.shape[-1]]), axis=0), # divide each pixel in the image by the maximum value in each channel (R, G, and B)
        label
    )

####### APP #######
st.title("Data preprocessing and inspection")
st.sidebar.header("Data inspection")
with st.spinner("Loading in the data"):
  train_ds, test_ds = initialize_data()
  fig = plot_10_images(train_ds)
st.write('Data loading done, example of training images')
st.pyplot(fig)
st.write('\n')

with st.spinner("Resizing the data and applying min-max scaling"):
  train_ds = train_ds.map(resize_images)
  test_ds = test_ds.map(resize_images)

  train_ds = train_ds.map(minmax_normalization)
  test_ds = test_ds.map(minmax_normalization)
st.write('Data preprocessing done, example of preprocessed training images')
st.pyplot(fig)
st.write('\n')


Writing pages/1_Data.py


In [ ]:
%%writefile pages/2_Model.py
import streamlit as st
from tensorflow.python.client import device_lib
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import os
####### FUNCTIONS #######
def select_device(prefer_gpu=True):
    local_device_protos = device_lib.list_local_devices()
    gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
    if (len(gpus) > 0) and prefer_gpu:
        return gpus[0]
    else:
        return [x.name for x in local_device_protos if x.device_type == 'CPU'][0]

def resize_images(image, label, image_width=40, image_height=40):
    """
    Resizes all images in a dataset to a uniform width and height.
    Also casts the images and labels to the float32 data type.
    """
    return (
        tf.cast(tf.image.resize_with_pad(tf.image.resize(image, (image_width, image_height)), image_width, image_height), tf.float32),
        tf.cast(label, tf.float32)
    )


def minmax_normalization(image, label):
    """Normalizes the pixel value range of an image to [0, 1] by dividing each pixel by the maximum value in each channel."""
    return (
        image / tf.math.reduce_max(tf.reshape(image, [-1, image.shape[-1]]), axis=0), # divide each pixel in the image by the maximum value in each channel (R, G, and B)
        label
    )

def initialize_data():
  device = select_device(prefer_gpu=True)
  builder = tfds.builder('malaria')
  builder.download_and_prepare()

  with tf.device(device):
    train_ds, test_ds = (
        builder.as_dataset(as_supervised=True, split="train[:80%]"),
        builder.as_dataset(as_supervised=True, split="train[-20%:]")
    )
  return train_ds, test_ds
def select_device(prefer_gpu=True):
    local_device_protos = device_lib.list_local_devices()
    gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
    if (len(gpus) > 0) and prefer_gpu:
        return gpus[0]
    else:
        return [x.name for x in local_device_protos if x.device_type == 'CPU'][0]


def run_model(train_ds, test_ds, epoch, batch_size):
  device = select_device(prefer_gpu=True)
  with tf.device(device):
    classifier = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(1,1), padding="same"),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=2, kernel_size=3, strides=(2,2), padding="same"),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    bce_loss = tf.keras.losses.BinaryCrossentropy()
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    accuracy_metric = tf.keras.metrics.BinaryAccuracy()

    with tf.device(device):
      classifier.compile(optimizer=sgd_optimizer, loss=bce_loss, metrics=[accuracy_metric])

    train_len = len(list(train_ds))
    test_len = len(list(test_ds))
    steps_per_epoch = train_len//batch_size
    test_steps = test_len//batch_size

    with tf.device(device):
      history = classifier.fit(
          train_ds.batch(batch_size).repeat(), # on which data to we want to train
          epochs=epoch, # how many epochs do we want to run
          steps_per_epoch=steps_per_epoch, # how many steps are in one epoch
          validation_data=test_ds.batch(batch_size).repeat(), # what test data do we want to use
          validation_steps=test_steps # how many steps do we need to take when testing
      )
  return(history)

####### APP #######
st.title('Model')
st.sidebar.header("Model training and inspection")

# Data loading
train_ds, test_ds = initialize_data()
train_ds = train_ds.map(resize_images)
test_ds = test_ds.map(resize_images)
train_ds = train_ds.map(minmax_normalization)
test_ds = test_ds.map(minmax_normalization)

with st.form("my_form"):
  st.write("Before starting the analysis, please choose the following values")
  epoch_val = st.slider("Number of epochs")
  batch_size = st.selectbox("Select batch size", (128,256), placeholder="Please choose a batch size")
  submitted = st.form_submit_button("Submit")

if submitted:
  with st.spinner("Training the model, please note this can take a while"):
    history = run_model(train_ds, test_ds, epoch_val, batch_size)

  plt.figure(figsize=(15, 8))
  plt.plot(history.epoch, history.history["binary_accuracy"], color="red")
  plt.plot(history.epoch, history.history["val_binary_accuracy"], color="blue")
  st.pyplot(plt)

Writing pages/2_Model.py


#### 🗒 **TASK** 🗒

Allow the users to select between the SGD and Adam optimizer.